# Anexo 3 - Hiperparámetros finales del modelo


In [1]:
import pandas as pd

df = pd.read_csv('../data/03_casos/df_52_cleaned.csv')

# Modelo de Prediccion de Hospitalizaciones en Menores de 5 Anos

## Objetivo del Notebook

Este notebook tiene como objetivo entrenar un modelo de machine learning (CatBoostRegressor) para predecir el numero de hospitalizaciones en menores de 5 anos debido a infecciones respiratorias agudas (IRA).

### Metodologia

1. **Datos de entrenamiento**: Se utilizan datos historicos desde 2006 hasta 2023
2. **Variable objetivo (target)**: hospitalizados_men5
3. **Algoritmo**: CatBoost (Categorical Boosting), un algoritmo de gradient boosting optimizado
4. **Horizonte de prediccion**: Generar predicciones para los anos 2024-2030

### Razon de uso de CatBoost

CatBoost es particularmente util para este caso porque:
- Maneja bien relaciones no lineales entre variables
- Es robusto ante overfitting gracias a su early stopping
- Tiene buen rendimiento con datasets de tamano mediano
- No requiere extensive feature engineering

In [2]:
df.head()

,ano,semana,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,sub_reg_nt,...,defunciones_60mas_lag_3_semana,defunciones_60mas_lag_4_semana,defunciones_60mas_lag_5_semana,defunciones_60mas_lag_6_semana,mes,bimestre,trimestre,cuatrimestre,semestre,pandemia_covid
0,2000,1,2168,16,0,9,0,0,0,52,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
1,2000,2,1986,13,0,7,0,0,0,52,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
2,2000,3,1836,15,0,5,0,0,0,52,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
3,2000,4,1459,16,0,4,0,0,0,52,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
4,2000,5,1431,11,0,4,0,0,0,52,...,0.0,0.0,0.0,0.0,2,1,1,1,1,0


## 1. Carga de Datos

Se carga el dataset limpio que contiene informacion semanal sobre casos de IRA, neumonias, hospitalizaciones y defunciones desde el ano 2000 hasta 2023.

El dataset incluye:
- Variables temporales: ano, semana
- Variables de casos: ira_no_neumonia, neumonias_men5, neumonias_60mas
- Variables de hospitalizaciones: hospitalizados_men5, hospitalizados_60mas
- Variables de defunciones: defunciones_men5, defunciones_60mas
- Variables derivadas: lags (rezagos temporales), variables de periodo (mes, bimestre, trimestre, etc.)

In [9]:
# --- IMPORTANTE: Asegurarse de que sean números ---
# Como vimos en tu problema de Power BI, si son texto, el orden falla.
# Esto garantiza que 'ano' y 'semana' sean tratados como números.
df['ano'] = pd.to_numeric(df['ano'])
df['semana'] = pd.to_numeric(df['semana'])

print("--- DataFrame de Ejemplo ---")
print(df)
print("\n" + "="*30 + "\n")


# === Método 1: Siguiendo tus pasos ===

# 1. Encontrar el año máximo
max_ano = df['ano'].max()

# 2. Filtrar el DataFrame para obtener solo las filas de ese año máximo
df_del_max_ano = df[df['ano'] == max_ano]

# 3. De ese DataFrame filtrado, encontrar la semana máxima
max_semana = df_del_max_ano['semana'].max()

print("--- Método 1 (Paso a paso) ---")
print(f"Año máximo: {max_ano}")
print(f"Semana máxima del año {max_ano}: {max_semana}")


# === Método 2: Ordenando los valores ===

# 1. Ordena el DataFrame por 'ano' y 'semana', ambos de mayor a menor
df_sorted = df.sort_values(by=['ano', 'semana'], ascending=False)

# 2. Obtiene la primera fila (que tendrá el 'ano' max y la 'semana' max)
latest_entry = df_sorted.iloc[0]

print("\n--- Método 2 (Ordenando) ---")
print(f"Año máximo: {latest_entry['ano']}")
print(f"Semana máxima de ese año: {latest_entry['semana']}")

--- DataFrame de Ejemplo ---
       ano  semana  ira_no_neumonia  neumonias_men5  neumonias_60mas  \
0     2000       1             2168              16                0   
1     2000       2             1986              13                0   
2     2000       3             1836              15                0   
3     2000       4             1459              16                0   
4     2000       5             1431              11                0   
...    ...     ...              ...             ...              ...   
1243  2023      48             1644              20               10   
1244  2023      49             1302              23               21   
1245  2023      50             1543              23               14   
1246  2023      51             1458              21               13   
1247  2023      52             1286              21               26   

      hospitalizados_men5  hospitalizados_60mas  defunciones_men5  \
0                       9            

## 2. Validacion y Conversion de Tipos de Datos

### Problema identificado

En integraciones previas con Power BI, se detecto que cuando las columnas 'ano' y 'semana' se interpretan como texto en lugar de numeros, el ordenamiento falla (ejemplo: "10" aparece antes que "2" en orden alfabetico).

### Solucion

Se fuerza la conversion de estas columnas a tipo numerico usando `pd.to_numeric()` para garantizar:
- Ordenamiento correcto de los datos temporales
- Calculos matematicos apropiados
- Correcta identificacion del ultimo periodo disponible

### Metodos de identificacion del ultimo periodo

Se demuestran dos metodos equivalentes para encontrar el ano y semana mas recientes en el dataset:

**Metodo 1 (Paso a paso)**:
1. Encontrar el ano maximo
2. Filtrar solo ese ano
3. Encontrar la semana maxima dentro de ese ano

**Metodo 2 (Ordenamiento)**:
1. Ordenar por ano y semana descendente
2. Tomar la primera fila

Ambos metodos garantizan obtener el periodo mas reciente del dataset.

## 3. Configuracion del Modelo

### Objetivo del modelo

Entrenar un modelo CatBoost usando datos del periodo 2006-2023 para predecir hospitalizaciones en menores de 5 anos (hospitalizados_men5) hasta el ano 2030.

### Variable objetivo (target)

**Target**: hospitalizados_men5

Esta variable representa el numero de hospitalizaciones de menores de 5 anos por semana. Se selecciono como target porque:
- Es un indicador critico de la severidad de las IRA en la poblacion infantil
- Permite anticipar la demanda de recursos hospitalarios
- Tiene una relacion directa con la planificacion de salud publica

In [3]:
# Preparar datos para entrenamiento
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import numpy as np

# Variables principales (sin incluir el target)
variables_principales = ['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
                         'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas']

# Target
target = 'hospitalizados_men5'

# Filtrar datos de 2006 a 2023 para entrenamiento
df_train = df[(df['ano'] >= 2006) & (df['ano'] <= 2023)].copy()

print(f"Datos de entrenamiento: {len(df_train)} registros")
print(f"Periodo: {df_train['ano'].min()}-{df_train['ano'].max()}")
print(f"Shape: {df_train.shape}")

Datos de entrenamiento: 936 registros
Periodo: 2006-2023
Shape: (936, 58)


### 3.1. Seleccion de Variables Predictoras

Se utilizan 8 variables principales como features para el modelo:

**Variables temporales**:
- ano, semana: Capturan patrones estacionales y tendencias temporales

**Variables de casos y severidad**:
- ira_no_neumonia: Casos de IRA que no derivaron en neumonia
- neumonias_men5: Neumonias en menores de 5 anos (indicador de gravedad)
- neumonias_60mas: Neumonias en mayores de 60 anos (patron epidemiologico)

**Variables de impacto en salud**:
- hospitalizados_60mas: Hospitalizaciones en adultos mayores
- defunciones_men5: Defunciones en menores de 5 anos
- defunciones_60mas: Defunciones en mayores de 60 anos

**Razon de la seleccion**: Estas variables tienen una relacion causal/correlacional directa con las hospitalizaciones infantiles, ya que reflejan la intensidad de la circulacion viral y la severidad de los casos en la poblacion.

### 3.2. Filtrado del Periodo de Entrenamiento

Se filtran datos desde 2006 hasta 2023 porque:
- Antes de 2006 podria haber cambios en los sistemas de reporte
- 2006-2023 proporciona 18 anos de datos (936 semanas), suficiente para capturar:
  - Variabilidad estacional (ciclos anuales)
  - Tendencias de largo plazo
  - Eventos atipicos (como la pandemia COVID-19)

In [4]:
# Separar features y target
X = df_train[variables_principales]
y = df_train[target]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo CatBoost
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=100
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

print("\nModelo entrenado exitosamente!")

0:	learn: 6.1997253	test: 5.9540056	best: 5.9540056 (0)	total: 148ms	remaining: 2m 27s
100:	learn: 2.9202205	test: 3.2663059	best: 3.2663059 (100)	total: 235ms	remaining: 2.09s
200:	learn: 2.3735181	test: 3.0874343	best: 3.0868875 (199)	total: 316ms	remaining: 1.26s
300:	learn: 2.0321449	test: 3.0407086	best: 3.0386070 (258)	total: 410ms	remaining: 953ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 3.038607041
bestIteration = 258

Shrink model to first 259 iterations.

Modelo entrenado exitosamente!


## 4. Entrenamiento del Modelo

### 4.1. Separacion de Datos

Se divide el dataset en:
- **80% para entrenamiento**: Datos que el modelo usa para aprender patrones
- **20% para validacion (test)**: Datos que el modelo NO ve durante el entrenamiento, usados para evaluar su capacidad de generalizacion

La semilla aleatoria (random_state=42) garantiza reproducibilidad de los resultados.

### 4.2. Hiperparametros de CatBoost

Los parametros configurados son:

- **iterations=1000**: Maximo numero de arboles de decision a construir
- **learning_rate=0.05**: Tasa de aprendizaje conservadora para evitar overfitting
- **depth=6**: Profundidad maxima de cada arbol (controla la complejidad del modelo)
- **loss_function='RMSE'**: Error cuadratico medio, apropiado para regresion
- **early_stopping_rounds=50**: Detiene el entrenamiento si no hay mejora en 50 iteraciones consecutivas

**Razon del early stopping**: Previene overfitting deteniendo el entrenamiento cuando el modelo deja de mejorar en el conjunto de validacion, incluso si no ha alcanzado el numero maximo de iteraciones.

In [5]:
# Evaluar el modelo
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2 = r2_score(y_test, y_pred_test)

print(f"Métricas del modelo:")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2: {r2:.4f}")

Métricas del modelo:
MAE: 2.2269
RMSE: 3.0386
R2: 0.7519


## 5. Evaluacion del Modelo

Se calculan tres metricas clave para evaluar el rendimiento del modelo:

### Metricas

1. **MAE (Mean Absolute Error - Error Absoluto Medio)**:
   - Promedio de las diferencias absolutas entre predicciones y valores reales
   - Interpretacion: En promedio, el modelo se equivoca por ±X hospitalizaciones

2. **RMSE (Root Mean Squared Error - Raiz del Error Cuadratico Medio)**:
   - Penaliza mas los errores grandes que el MAE
   - Util para identificar si hay predicciones muy alejadas de la realidad

3. **R² (Coeficiente de Determinacion)**:
   - Indica que proporcion de la variabilidad del target es explicada por el modelo
   - Rango: 0 a 1, donde 1 es perfecto
   - Un R² de 0.75 significa que el modelo explica el 75% de la varianza

### Interpretacion de resultados

- Si MAE y RMSE son similares: los errores son consistentes
- Si RMSE >> MAE: hay algunos errores muy grandes (outliers en predicciones)
- Si R² > 0.7: el modelo tiene buen poder predictivo

In [ ]:
# Generar fechas futuras (2024-2025)
# Calcular promedios históricos por semana para las variables predictoras
promedios_por_semana = df_train.groupby('semana')[
    ['ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas', 
     'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas']
].mean()

# Crear DataFrame para predicciones futuras
years_future = [2024, 2025, 2026, 2027, 2028, 2029, 2030]
semanas = list(range(1, 53))  # 52 semanas por año

future_data = []
for year in years_future:
    for semana in semanas:
        row = {'ano': year, 'semana': semana}
        # Usar promedios históricos para cada semana del año
        for col in ['ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas', 
                    'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas']:
            row[col] = promedios_por_semana.loc[semana, col]
        future_data.append(row)

df_future = pd.DataFrame(future_data)
print(f"Datos futuros generados: {len(df_future)} registros (años 2024-2025)")
print(df_future.head())

## 6. Generacion de Datos Futuros para Prediccion

### Desafio

Para predecir hospitalizaciones futuras (2024-2030), necesitamos valores de las variables predictoras (ira_no_neumonia, neumonias_men5, etc.) que aun no existen.

### Estrategia: Promedios Historicos por Semana

Se calcula el promedio historico de cada variable para cada semana del ano (semanas 1-52) basandose en los datos 2006-2023.

**Razon de este enfoque**:
- Las IRA tienen un patron estacional marcado (mas casos en invierno)
- Usar el promedio de la semana 10 de todos los anos historicos es una aproximacion razonable de lo que podria ocurrir en la semana 10 de 2024
- Este metodo captura la estacionalidad sin hacer suposiciones sobre tendencias futuras

### Limitaciones

- Asume que los patrones historicos se mantendran
- No considera eventos extraordinarios futuros (nuevas pandemias, cambios climaticos, etc.)
- No captura tendencias de largo plazo (aumento/disminucion poblacional)

### Construccion del DataFrame Futuro

Se crea un dataset con:
- Todas las semanas (1-52) de cada ano (2024-2030)
- Valores de features basados en promedios historicos de esa semana
- Total: 7 anos × 52 semanas = 364 registros

In [ ]:
# Realizar predicciones para 2024-2025
X_future = df_future[variables_principales]
predicciones = model.predict(X_future)

# Agregar predicciones al DataFrame futuro
df_future['hospitalizados_men5_prediccion'] = predicciones

print(f"Predicciones realizadas: {len(predicciones)} valores")
print("\nPrimeras 10 predicciones:")
print(df_future[['ano', 'semana', 'hospitalizados_men5_prediccion']].head(10))
print("\nEstadísticas de predicciones:")
print(df_future['hospitalizados_men5_prediccion'].describe())

## 7. Realizacion de Predicciones Futuras

### Proceso

1. Se toman las variables predictoras del DataFrame futuro (X_future)
2. El modelo entrenado genera predicciones de hospitalizados_men5 para cada semana
3. Las predicciones se agregan como nueva columna al DataFrame

### Validacion de Resultados

Se muestran:
- Las primeras 10 predicciones para inspeccion visual
- Estadisticas descriptivas (min, max, media, desviacion) para detectar valores anomalos

**Que verificar**:
- Las predicciones deben estar en un rango razonable (no negativos, no extremadamente altos)
- Debe haber variabilidad entre semanas (reflejando estacionalidad)
- Los valores promedio deben ser coherentes con los datos historicos

In [ ]:
# Guardar predicciones en CSV
output_path = '../data/03_casos/df_52_cleaned_predicciones.csv'
df_future.to_csv(output_path, index=False)

print(f"\n✓ Predicciones guardadas exitosamente en: {output_path}")
print(f"  - Total de registros: {len(df_future)}")
print(f"  - Años incluidos: {df_future['ano'].min()} - {df_future['ano'].max()}")
print(f"  - Columnas: {list(df_future.columns)}")

## 8. Exportacion de Resultados

### Guardado de Predicciones

Las predicciones se guardan en formato CSV para:
- Facilitar la integracion con otras herramientas (Power BI, Excel, bases de datos)
- Mantener un registro de las predicciones generadas
- Permitir analisis posteriores

### Contenido del Archivo

El archivo CSV incluye:
- Variables temporales (ano, semana)
- Variables predictoras utilizadas (con valores promedio historicos)
- Columna de predicciones: hospitalizados_men5_prediccion

### Uso Posterior

Este archivo puede ser:
- Cargado en Power BI para visualizacion de tendencias futuras
- Combinado con datos reales cuando esten disponibles para validar precision
- Utilizado para planificacion de recursos hospitalarios